In [1]:
import pandas as pd



In [7]:
pip install pandas pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 61.6 MB/s eta 0:00:000m eta 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
df=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',engine="pyarrow")

In [8]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [7]:
len(df.columns)

19

In [10]:
df['tpep_pickup_datetime']

0         2023-01-01 00:32:10
1         2023-01-01 00:55:08
2         2023-01-01 00:25:04
3         2023-01-01 00:03:48
4         2023-01-01 00:10:29
                  ...        
3066761   2023-01-31 23:58:34
3066762   2023-01-31 23:31:09
3066763   2023-01-31 23:01:05
3066764   2023-01-31 23:40:00
3066765   2023-01-31 23:07:32
Name: tpep_pickup_datetime, Length: 3066766, dtype: datetime64[us]

In [11]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

print("Standard deviation of trip durations (minutes):", df['duration'].std())

Standard deviation of trip durations (minutes): 42.59435124195458


In [16]:
df_clean=df[(df['duration']>=1) & (df['duration']<=60)]
total=len(df)
fraction=len(df_clean)/total
print("fraction :",fraction*100,"%")

fraction : 98.1220282212598 %


In [23]:
from sklearn.feature_extraction import DictVectorizer

# Select the relevant columns and convert IDs to strings
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Convert the DataFrame to a list of dictionaries
data_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Initialize and fit the DictVectorizer
dv = DictVectorizer(sparse=True)
feature_matrix = dv.fit_transform(data_dicts)

# Determine the dimensionality of the feature matrix
num_columns = feature_matrix.shape[1]
print(f"The dimensionality of the feature matrix is: {num_columns}")


The dimensionality of the feature matrix is: 518


In [24]:
unique_pu = df['PULocationID'].nunique()
unique_do = df['DOLocationID'].nunique()
print(unique_pu + unique_do)


518


In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load data
df = pd.read_parquet(
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet',
    engine="pyarrow"
)

# Compute trip duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Convert IDs to strings
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Vectorize
dv = DictVectorizer()
train_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"Train RMSE: {rmse:.2f}")


Train RMSE: 7.65


In [ ]:
# Load validation data
df_val = pd.read_parquet(
    'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet',
    engine="pyarrow"
)

# Compute duration
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

# Convert IDs to strings
df_val['PULocationID'] = df_val['PULocationID'].astype(str)
df_val['DOLocationID'] = df_val['DOLocationID'].astype(str)

# Vectorize using the previously fitted DictVectorizer (dv)
val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(val_dicts)  # IMPORTANT: use transform, not fit_transform
y_val = df_val['duration'].values

# Predict and evaluate
y_pred_val = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Validation RMSE: {rmse_val:.2f}")
